# Blockchain

La blockchain o cadena de bloques es una serie de documentos en donde cada uno esta relacionado con el anterior mediante su hash. Esto es, el segundo bloque tendra el hash del primero, el tercero tendra el hash del segundo y asi sucesivamente.


# Ejercicio:

Publicar tres bloques cada uno en un archivo diferente. Cada bloque del archivo comenzara con Bloque N y finalizara cuando finalice el archivo y contendra sus respectivas transacciones escritas en texto plano y firmadas por cada participante como indica el ejemplo. Finalmente cada bloque debe ser firmado por el administrador que garantiza que es valido.

Publicarlos en github u otro medio para que todos los participantes puedan ver los bloques y descargarlos.

Crear una funcion que verifique que cada bloque es valido.

Ejemplo de bloques:

```
Bloque 0:
(hash = 000000000000000)
Se crean 10 monedas para Alicia
(firma Admin)
----------
Bloque 1:
(hash Bloque 0)
Alicia le envia 3 monedas a Bob (firma Alicia)
(firma Admin)
----------
Bloque 2:
(hash Bloque 1)
Alicia le envia 4 monedas a Carolina (firma Alicia)
Bob le envia 2 monedas a Carolina (firma Bob)
(firma Admin)
----------
```


In [3]:
!apt-get update
!apt-get install -y build-essential python3-dev libgmp3-dev libssl-dev
!pip install bitcoinlib

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,199 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,639 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

In [6]:
import hashlib
from bitcoinlib.keys import Key, sign, verify

# Función simple para crear nuevas claves
def crear_claves():
    clave = Key()
    return clave.secret, clave.public()

# Función para crear hash SHA256
def crear_hash(mensaje):
    return hashlib.sha256(mensaje.encode()).hexdigest()

# Función para firmar un mensaje
def firmar(mensaje, clave_privada):
    return sign(mensaje.encode(), clave_privada)

# Función para verificar una firma
def verificar(mensaje, firma, clave_publica):
    return verify(mensaje.encode(), firma, clave_publica)

# Función para guardar en archivo
def guardar(contenido, archivo):
    with open(archivo, 'w') as f:
        f.write(contenido)

# Función para verificar si una transacción es posible
def verificar_transaccion(saldos, emisor, receptor, cantidad):
    if emisor not in saldos:
        return False, f"Error: {emisor} no tiene una cuenta"
    if saldos[emisor] < cantidad:
        return False, f"Error: {emisor} no tiene suficientes monedas (tiene {saldos[emisor]}, intenta enviar {cantidad})"
    return True, "Transacción válida"

# Función principal que maneja todo el proceso
def main():
    # Crear claves para todos
    clave_privada_admin, clave_publica_admin = crear_claves()
    clave_privada_marco, clave_publica_marco = crear_claves()
    clave_privada_jorge, clave_publica_jorge = crear_claves()
    clave_privada_alberto, clave_publica_alberto = crear_claves()
    clave_privada_carolina, clave_publica_carolina = crear_claves()

    # Diccionario para mantener el registro de saldos
    saldos = {
        "Marco": 0,
        "Jorge": 0,
        "Alberto": 0,
        "Carolina": 0
    }

    # Bloque 0: Crear monedas para Marco
    mensaje_0 = "Se crean 10 monedas para Marco"
    firma_admin_0 = firmar(mensaje_0, clave_privada_admin)
    saldos["Marco"] += 10  # Actualizar saldo de Marco

    bloque_0 = f"""Bloque 0:
    --------------------------
    Hash Anterior: 000000000000000
    Transacción: {mensaje_0}
    Firma Admin: {firma_admin_0}
    """

    guardar(bloque_0, "bloque_0.txt")
    hash_bloque_0 = crear_hash(bloque_0)

    # Bloque 1: Marco envía monedas a Jorge
    # Verificar si la transacción es posible
    es_posible, mensaje = verificar_transaccion(saldos, "Marco", "Jorge", 7)
    if not es_posible:
        print(mensaje)
        return

    mensaje_1 = "Marco envía 7 monedas a Jorge"
    firma_marco = firmar(mensaje_1, clave_privada_marco)
    firma_admin_1 = firmar(mensaje_1, clave_privada_admin)

    # Actualizar saldos
    saldos["Marco"] -= 7
    saldos["Jorge"] += 7

    bloque_1 = f"""Bloque 1:
    ----------------------------
    Hash Anterior: {hash_bloque_0}
    Transacción: {mensaje_1}
    Firma Marco: {firma_marco}
    Firma Admin: {firma_admin_1}
    """

    guardar(bloque_1, "bloque_1.txt")
    hash_bloque_1 = crear_hash(bloque_1)

    # Bloque 2: Marco envía monedas a Alberto y Carolina
    # Verificar ambas transacciones
    es_posible_1, mensaje_1 = verificar_transaccion(saldos, "Marco", "Alberto", 2)
    es_posible_2, mensaje_2 = verificar_transaccion(saldos, "Marco", "Carolina", 1)

    if not (es_posible_1 and es_posible_2):
        print(f"Error en transacciones del bloque 2:\n{mensaje_1}\n{mensaje_2}")
        return

    mensaje_2a = "Marco envía 2 monedas a Alberto"
    mensaje_2b = "Marco envía 1 monedas a Carolina"

    firma_marco_2 = firmar(mensaje_2a, clave_privada_marco)
    firma_marco_3 = firmar(mensaje_2b, clave_privada_marco)
    firma_admin_2 = firmar(mensaje_2a + mensaje_2b, clave_privada_admin)

    # Actualizar saldos
    saldos["Marco"] -= 3
    saldos["Alberto"] += 2
    saldos["Carolina"] += 1

    bloque_2 = f"""Bloque 2:
    ----------------------------
    Hash Anterior: {hash_bloque_1}
    Transacción 1: {mensaje_2a}
    Firma Marco: {firma_marco_2}
    Transacción 2: {mensaje_2b}
    Firma Marco: {firma_marco_3}
    Firma Admin: {firma_admin_2}
    """

    guardar(bloque_2, "bloque_2.txt")

    print("Bloques creados y guardados correctamente")
    print("\nSaldos finales:")
    for usuario, saldo in saldos.items():
        print(f"{usuario}: {saldo} monedas")

if __name__ == "__main__":
    main()

Bloques creados y guardados correctamente

Saldos finales:
Marco: 0 monedas
Jorge: 7 monedas
Alberto: 2 monedas
Carolina: 1 monedas
